## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-21-14-53-34 +0000,nypost,Online gambling markets field millions in prop...,https://nypost.com/2026/02/21/us-news/trumps-s...
1,2026-02-21-14-50-46 +0000,nypost,Sex predator trans inmate asks to be moved to ...,https://nypost.com/2026/02/21/world-news/sex-p...
2,2026-02-21-14-50-12 +0000,nyt,Bridge Owner Donated $1 Million to MAGA Group ...,https://www.nytimes.com/2026/02/20/us/politics...
3,2026-02-21-14-47-23 +0000,nyt,Divide Among Supreme Court’s Conservatives Cou...,https://www.nytimes.com/2026/02/21/us/politics...
4,2026-02-21-14-46-51 +0000,nyt,Blizzard Warnings Issued for Swath of East Coa...,https://www.nytimes.com/2026/02/21/us/blizzard...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2423/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,73
34,court,47
33,supreme,42
100,tariffs,33
331,tariff,29


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...,321
66,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,253
209,2026-02-20-20-56-39 +0000,wapo,"Trump imposes new tariffs, denounces Supreme C...",https://www.washingtonpost.com/national-securi...,248
162,2026-02-20-23-26-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,247
274,2026-02-20-17-23-09 +0000,nypost,Democrats giddy over Trump Supreme Court tarif...,https://nypost.com/2026/02/20/us-news/democrat...,243


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,321,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...
241,70,2026-02-20-19-18-02 +0000,latimes,"Trump calls justices 'fools,' announces new 10...",https://www.latimes.com/politics/story/2026-02...
145,58,2026-02-21-00-49-41 +0000,nypost,Los Angeles small business owner takes wait-an...,https://nypost.com/2026/02/20/business/los-ang...
290,40,2026-02-20-16-34-59 +0000,nyt,Trump says he’s considering a limited strike o...,https://www.nytimes.com/live/2026/02/20/us/tru...
301,38,2026-02-20-15-58-01 +0000,nypost,The government agency relying on Wall Street t...,https://nypost.com/2026/02/20/business/develop...
231,33,2026-02-20-19-56-29 +0000,nyt,Former Prince Andrew’s Arrest Upends Royal Fam...,https://www.nytimes.com/2026/02/20/world/europ...
17,32,2026-02-21-13-53-38 +0000,nypost,Shocking video shows group of NYC teens climbi...,https://nypost.com/2026/02/21/us-news/shock-vi...
6,32,2026-02-21-14-21-18 +0000,nypost,DOJ axed NY judge who handed out more asylum c...,https://nypost.com/2026/02/21/us-news/youre-fi...
11,30,2026-02-21-14-08-30 +0000,startribune,Live: U.S. women’s curling team faces Canada f...,https://www.startribune.com/2026-winter-olympi...
14,29,2026-02-21-13-58-07 +0000,nypost,How savvy GOPers plan to grill Bill and Hillar...,https://nypost.com/2026/02/21/us-news/in-the-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
